"Deskew delle immagini"

Consiste nel ruotare e raddrizzare le immagini in modo da non riscontrare problemi durante la fase successiva di feature extraction.

In [50]:
import cv2 as cv
import numpy as np

def deskew(img):
    # Assicurati che l'immagine sia in scala di grigi
    if len(img.shape) == 3:  # 3 canali, quindi è a colori
        img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    # Controlla se l'immagine è in scala di grigi
    if len(img.shape) != 2:
        raise ValueError("L'immagine deve essere in scala di grigi.")
    
    m = cv.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()

    skew = m['mu11'] / m['mu02']
    M = np.float32([[1, skew, 0], [0, 1, 0]])
    rows, cols = img.shape
    deskewed_img = cv.warpAffine(img, M, (cols, rows), flags=cv.WARP_INVERSE_MAP | cv.INTER_LINEAR)
    return deskewed_img

COMMENTO DA QUI ALLA FINE DEL FILE

"Feature extraction con tecnica HOG"

In [61]:
from skimage.feature import hog
from skimage import io, color
import matplotlib.pyplot as plt

def hog_feature_extraction(image_path):
    image = cv.imread(image_path)
    #deskewed_image = deskew(image)

    if len(image.shape) != 2:
        raise ValueError("L'immagine non è in scala di grigi.")

    hog_descriptor, hog_image = hog(image, 
                         pixels_per_cell = (8, 8),
                         cells_per_block = (2, 2),
                         visualize = True)
    
    """ # Display the original image and HOG image for debugging
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)

    ax1.axis('off')
    ax1.imshow(gray_image, cmap=plt.cm.gray)
    ax1.set_title('Input Image')

    ax2.axis('off')
    ax2.imshow(hog_image, cmap=plt.cm.gray)
    ax2.set_title('HOG Gradient Image')

    plt.show()"""

    return hog_descriptor

In [62]:
import os

image_dir = "data/emnist_train_images/1"
image_names = []
descriptor = []

for filename in os.listdir(image_dir):
    if(filename.endswith(".png")):
        image_path = os.path.join(image_dir, filename)
        hog_descriptor = hog_feature_extraction(image_path)
        image_names.append(filename)
        descriptor.append(hog_descriptor)

result_matrix = np.column_stack((image_names, descriptor))

for row in result_matrix:
    print(row[0]+" -> "+ row[1])


ValueError: L'immagine non è in scala di grigi.